# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 1:12PM,244286,19,STR08162022-1,ACG North America LLC,Released
1,Aug 16 2022 1:04PM,244285,10,SNAP4515370005,"Snap Medical Industries, LLC",Released
2,Aug 16 2022 1:04PM,244285,10,SNAP4515370020,"Snap Medical Industries, LLC",Released
3,Aug 16 2022 1:04PM,244285,10,SNAP816221,"Snap Medical Industries, LLC",Released
4,Aug 16 2022 1:04PM,244285,10,SNAP816222,"Snap Medical Industries, LLC",Released
5,Aug 16 2022 1:04PM,244285,10,SNAPF2508201,"Snap Medical Industries, LLC",Released
6,Aug 16 2022 1:04PM,244285,10,SNAPNE2057530685,"Snap Medical Industries, LLC",Released
7,Aug 16 2022 1:04PM,244285,10,SNAPR336388264,"Snap Medical Industries, LLC",Released
8,Aug 16 2022 12:59PM,244284,16,AIC-379504,American International Chemical,Released
9,Aug 16 2022 12:49PM,244272,10,MTR-220246,"Senseonics, Incorporated",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,244282,Released,1
44,244283,Released,1
45,244284,Released,1
46,244285,Released,7
47,244286,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244282,NaN,NaN,1.0
244283,NaN,NaN,1.0
244284,NaN,NaN,1.0
244285,NaN,NaN,7.0
244286,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244183,0.0,0.0,1.0
244189,0.0,0.0,1.0
244191,0.0,0.0,1.0
244192,0.0,0.0,12.0
244194,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244183,0,0,1
244189,0,0,1
244191,0,0,1
244192,0,0,12
244194,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244183,0,0,1
1,244189,0,0,1
2,244191,0,0,1
3,244192,0,0,12
4,244194,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244183,,,1
1,244189,,,1
2,244191,,,1
3,244192,,,12
4,244194,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 1:12PM,244286,19,ACG North America LLC
1,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC"
8,Aug 16 2022 12:59PM,244284,16,American International Chemical
9,Aug 16 2022 12:49PM,244272,10,"Senseonics, Incorporated"
10,Aug 16 2022 12:48PM,244283,12,"NBTY Global, Inc."
11,Aug 16 2022 12:46PM,244282,12,"SD Head USA, LLC"
12,Aug 16 2022 12:40PM,244281,12,Hush Hush
13,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC
19,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc."
26,Aug 16 2022 12:24PM,244275,16,Seqirus


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 16 2022 1:12PM,244286,19,ACG North America LLC,,,1
1,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC",,,7
2,Aug 16 2022 12:59PM,244284,16,American International Chemical,,,1
3,Aug 16 2022 12:49PM,244272,10,"Senseonics, Incorporated",,,1
4,Aug 16 2022 12:48PM,244283,12,"NBTY Global, Inc.",,,1
5,Aug 16 2022 12:46PM,244282,12,"SD Head USA, LLC",,,1
6,Aug 16 2022 12:40PM,244281,12,Hush Hush,,,1
7,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC,,,6
8,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc.",,,7
9,Aug 16 2022 12:24PM,244275,16,Seqirus,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 1:12PM,244286,19,ACG North America LLC,1,,
1,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC",7,,
2,Aug 16 2022 12:59PM,244284,16,American International Chemical,1,,
3,Aug 16 2022 12:49PM,244272,10,"Senseonics, Incorporated",1,,
4,Aug 16 2022 12:48PM,244283,12,"NBTY Global, Inc.",1,,
5,Aug 16 2022 12:46PM,244282,12,"SD Head USA, LLC",1,,
6,Aug 16 2022 12:40PM,244281,12,Hush Hush,1,,
7,Aug 16 2022 12:33PM,244279,10,MedStone Pharma LLC,6,,
8,Aug 16 2022 12:31PM,244277,10,"Methapharm, Inc.",7,,
9,Aug 16 2022 12:24PM,244275,16,Seqirus,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 1:12PM,244286,19,ACG North America LLC,1,,
1,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC",7,,
2,Aug 16 2022 12:59PM,244284,16,American International Chemical,1,,
3,Aug 16 2022 12:49PM,244272,10,"Senseonics, Incorporated",1,,
4,Aug 16 2022 12:48PM,244283,12,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 1:12PM,244286,19,ACG North America LLC,1.0,NaN,NaN
1,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC",7.0,NaN,NaN
2,Aug 16 2022 12:59PM,244284,16,American International Chemical,1.0,NaN,NaN
3,Aug 16 2022 12:49PM,244272,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Aug 16 2022 12:48PM,244283,12,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 1:12PM,244286,19,ACG North America LLC,1.0,0.0,0.0
1,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC",7.0,0.0,0.0
2,Aug 16 2022 12:59PM,244284,16,American International Chemical,1.0,0.0,0.0
3,Aug 16 2022 12:49PM,244272,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Aug 16 2022 12:48PM,244283,12,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2931054,70.0,17.0,0.0
12,732846,3.0,0.0,0.0
15,732653,44.0,35.0,45.0
16,1709690,7.0,0.0,0.0
19,1221365,4.0,6.0,0.0
20,1465437,60.0,1.0,0.0
21,732583,3.0,0.0,0.0
22,732772,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2931054,70.0,17.0,0.0
1,12,732846,3.0,0.0,0.0
2,15,732653,44.0,35.0,45.0
3,16,1709690,7.0,0.0,0.0
4,19,1221365,4.0,6.0,0.0
5,20,1465437,60.0,1.0,0.0
6,21,732583,3.0,0.0,0.0
7,22,732772,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,70.0,17.0,0.0
1,12,3.0,0.0,0.0
2,15,44.0,35.0,45.0
3,16,7.0,0.0,0.0
4,19,4.0,6.0,0.0
5,20,60.0,1.0,0.0
6,21,3.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,70.0
1,12,Released,3.0
2,15,Released,44.0
3,16,Released,7.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0
Executing,17.0,0.0,35.0,0.0,6.0,1.0,0.0,0.0
Released,70.0,3.0,44.0,7.0,4.0,60.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0
1,Executing,17.0,0.0,35.0,0.0,6.0,1.0,0.0,0.0
2,Released,70.0,3.0,44.0,7.0,4.0,60.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0
1,Executing,17.0,0.0,35.0,0.0,6.0,1.0,0.0,0.0
2,Released,70.0,3.0,44.0,7.0,4.0,60.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()